In [ ]:
#!wget https://github.com/nvs543/AIML_B11_NLP/blob/c1666ca7a183eece7073c4c7e0946b8f46bfe1ed/bbc-fulltext.zip
#!wget https://github.com/nvs543/AIML_B11_NLP/blob/796491820603626de2fcfec6ae69fef3937a1866/bbc.zip
!wget /content/drive/MyDrive/Colab Notebooks/bbc.zip

/content/drive/MyDrive/Colab: Scheme missing.
--2022-12-02 09:22:25--  http://notebooks/bbc.zip
Resolving notebooks (notebooks)... failed: Name or service not known.
wget: unable to resolve host address ‘notebooks’


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
from urllib import request
import re
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import os
import glob
import string

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
!pip3 install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 27.2 MB/s 
     |████████████████████████████████| 588.3 MB 22 kB/s 
     |████████████████████████████████| 439 kB 68.9 MB/s 
     |████████████████████████████████| 6.0 MB 49.6 MB/s 
     |████████████████████████████████| 1.7 MB 58.7 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.

In [2]:
import numpy as np
import pandas as pd
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Dropout, Activation
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix , classification_report

In [3]:
bbc_path = "/content/drive/MyDrive/Colab Notebooks/bbc/"

In [4]:
def readFiles(path,clsId):
  fileNames = []
  fNames = []
  articles = []
  clsIds = []
  #print(glob.glob(path+"*.txt"))
  filenames=os.listdir(path)
  
  for ef in filenames:
    f = open(path+ef).readlines()
    articles.append(" ".join(f))
    clsIds.append(clsId)
    fNames.append(ef)
  
  return fNames,articles,clsIds
  

In [5]:
classNames = ["business","entertainment","politics","sport","tech"]
allArticles = []
allClsIds = []
allFileNames = []
for idx in range(len(classNames)):
  p = bbc_path+classNames[idx]+"/"
  fn,ad,cid = readFiles(p,idx)
  allArticles = allArticles + ad
  allClsIds = allClsIds + cid
  allFileNames = allFileNames + fn
 
print(len(allArticles),len(allClsIds),len(allFileNames))

2225 2225 2225


In [6]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [7]:
for i in range(len(allArticles)):
  allArticles[i] = allArticles[i].lower() #convert to lower case
  allArticles[i] = re.sub('<.*?>',' ',allArticles[i]) # remove html tags
  allArticles[i] = re.sub('[^a-zA-Z]',' ',allArticles[i]) #remove non-alphabet
  allArticles[i] = re.sub(r'\s+[a-zA-Z]\s+',' ',allArticles[i]) #remove single character
  allArticles[i] = re.sub(r'\s+', ' ',allArticles[i]) #remvoe multiple spaces
  allArticles[i] = REPLACE_BY_SPACE_RE.sub(' ',allArticles[i]) # replace REPLACE_BY_SPACE_RE symbols by space in text
  allArticles[i] = BAD_SYMBOLS_RE.sub('',allArticles[i]) #remove symbols that are in BAD_SYMBOLS from text
  allArticles[i] = allArticles[i].replace('x','') #remove x in text
  allArticles[i] = ' '.join(word for word in allArticles[i].split() if word not in STOPWORDS) # remove stop words

In [8]:
allArticles[0]

'us interest rate rise epected us interest rates epected rise fifth time since june following us federal reserve latest rate setting meeting later tuesday borrowing costs tipped rise quarter percentage point move comes recovery us economy world biggest shows signs robustness sustainability dollar record breaking decline meanwhile spooked markets along high oil prices raised concerns pace inflation seeing evidence inflation moving higher said ken kim analyst stone mccarthy research risk actually happening mr kim added borrowing costs could rise fed said move measured way combat price growth lift interest rates year lows prompted sluggish us global growth economic picture looking rosy fed implemented quarter percentage point rises june august september november although us economy grew annual rate three months september analysts warn fed careful move aggressively take wind recovery sails earlier month figures showed job creation still weak consumer confidence subdued think fed feels fair

In [9]:
#df with word2vec
cols = ['article', 'labels']
d = {'article':allArticles, 'labels':allClsIds}
df = pd.DataFrame(data=d, columns=cols)
df.head()

,article,labels
0,us interest rate rise epected us interest rate...,0
1,uk economy facing major risks uk manufacturing...,0
2,ask jeeves tips online ad revival ask jeeves b...,0
3,us company admits benin bribery us defence tel...,0
4,ticking budget facing us budget proposals laid...,0


In [10]:
val_counts = df['labels'].value_counts()
val_counts

3    511
0    510
2    417
4    401
1    386
Name: labels, dtype: int64

In [11]:
df_0 = df[df['labels']==0]
df_1 = df[df['labels']==1]
df_2 = df[df['labels']==2]
df_3 = df[df['labels']==3]
df_4 = df[df['labels']==4]

print(df_0.shape)
print(df_1.shape)
print(df_2.shape)
print(df_3.shape)
print(df_4.shape)

(510, 2)
(386, 2)
(417, 2)
(511, 2)
(401, 2)


In [12]:
df_0_downsampled = df_0.sample(df_1.shape[0])
df_2_downsampled = df_2.sample(df_1.shape[0])
df_3_downsampled = df_3.sample(df_1.shape[0])
df_4_downsampled = df_4.sample(df_1.shape[0])

print(df_0_downsampled.shape)
print(df_2_downsampled.shape)
print(df_3_downsampled.shape)
print(df_4_downsampled.shape)

(386, 2)
(386, 2)
(386, 2)
(386, 2)


In [13]:
df_balanced = pd.concat([df_0_downsampled,df_1,df_2_downsampled,df_3_downsampled,df_4_downsampled])
df_balanced.shape

(1930, 2)

In [14]:
df_balanced.head()

,article,labels
479,iraqi voters turn economic issues beyond despe...,0
134,korea spending boost economy south korea boost...,0
260,iraq invite phone licence bids iraq invite bid...,0
109,malaysia lifts islamic bank limit malaysia cen...,0
435,uk interest rates held bank england left inter...,0


In [15]:
df_balanced['labels'].value_counts()

0    386
1    386
2    386
3    386
4    386
Name: labels, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanced['article'],df_balanced['labels'],stratify=df_balanced['labels'])
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(1447,) (483,) (1447,) (483,)


In [19]:
print("Train split by classes\n",y_train.value_counts(),"\n\n", "Test split by classes\n",y_test.value_counts())

Train split by classes
 1    290
4    290
0    289
3    289
2    289
Name: labels, dtype: int64 

 Test split by classes
 2    97
0    97
3    97
1    96
4    96
Name: labels, dtype: int64


## BERT - Bidirectional Encoder Representations from Transformers

In [29]:

# Downloading preprocessing files and model from the TF models repository. This will take more time for dowloading the files
preprocessing_layer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [44]:
def build_classifier_model():
  text_input = keras.layers.Input(shape=(), dtype=tf.string, name='text')
  
  #pre-processing the text input
  encoded_inputs = preprocessing_layer(text_input)

  #pass encoded_inputs through the BERT model
  outputs = bert_encoder(encoded_inputs);
  
  #Model returns embedding of the entire dataset
  net = outputs['pooled_output']
  
  #pass the entire training data embeddings to the dropout layer
  net = keras.layers.Dropout(0.2, name="dropout")(net)

  #dense layer
  net = keras.layers.Dense(5, activation='softmax', name='output')(net)
  results = keras.Model(text_input, net)
  return results


In [45]:
model = build_classifier_model()
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                

In [46]:
#compile the model
model.compile(optimizer ='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])



In [47]:
history = model.fit(X_train, y_train, epochs = 15)

Epoch 1/15
46/46 [==============================] - 17s 315ms/step - loss: 1.5349 - accuracy: 0.3158
Epoch 2/15
46/46 [==============================] - 15s 333ms/step - loss: 1.3055 - accuracy: 0.4824
Epoch 3/15
46/46 [==============================] - 15s 334ms/step - loss: 1.1442 - accuracy: 0.5874
Epoch 4/15
46/46 [==============================] - 15s 326ms/step - loss: 1.0192 - accuracy: 0.6821
Epoch 5/15
46/46 [==============================] - 15s 330ms/step - loss: 0.9271 - accuracy: 0.7360
Epoch 6/15
46/46 [==============================] - 15s 329ms/step - loss: 0.8359 - accuracy: 0.7719
Epoch 7/15
46/46 [==============================] - 15s 328ms/step - loss: 0.7742 - accuracy: 0.8086
Epoch 8/15
46/46 [==============================] - 15s 328ms/step - loss: 0.7045 - accuracy: 0.8189
Epoch 9/15
46/46 [==============================] - 15s 326ms/step - loss: 0.6614 - accuracy: 0.8314
Epoch 10/15
46/46 [==============================] - 15s 328ms/step - loss: 0.6405 - accura

In [48]:
model.evaluate(X_test,y_test)

16/16 [==============================] - 6s 307ms/step - loss: 0.4594 - accuracy: 0.9151


[0.45935124158859253, 0.9151138663291931]

In [52]:
#download trained model to local machine
model.save("/content/2A_trained_model_2/2A_BBC_bert.h5")

In [59]:
model_pt = tf.keras.models.load_model(
    "/content/2A_trained_model"
)

In [60]:
model_pt.evaluate(X_test,y_test)

16/16 [==============================] - 6s 300ms/step - loss: 0.4594 - accuracy: 0.9151


[0.45935124158859253, 0.9151138663291931]